In [ ]:
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
import SimpleITK as sitk

In [ ]:
import ct_projector.projector.cupy as ct_projector
import ct_projector.projector.cupy.cone as ct_cone

In [ ]:
# load a sample CT image
filename = './3.nii.gz'
ct = sitk.ReadImage(filename)
spacing = ct.GetSpacing()
img = sitk.GetArrayFromImage(ct)

# convert image from HU to attenuation coefficient
# This is the approximate relationship
img = (img.astype(np.float32) + 1024) / 1000 * 0.019
img[img < 0] = 0

# also convert to image to our projector dimension batch, z, y, x
img = img[np.newaxis,...]
img = img[:, ::-1, ...]
spacing = np.array(spacing[::-1])

In [ ]:
# show the ct images
plt.figure(figsize = (12,4))
plt.subplot(131); plt.imshow(img[0, img.shape[1]//2, ...], 'gray', aspect=spacing[1] / spacing[2])
plt.subplot(132); plt.imshow(img[0, :, img.shape[2]//2, :], 'gray', aspect=spacing[0] / spacing[2])
plt.subplot(133); plt.imshow(img[0, ..., img.shape[3]//2], 'gray', aspect=spacing[0] / spacing[1])

In [ ]:
# setup the projector
projector = ct_projector.ct_projector()
projector.from_file('./projector.cfg')
projector.nx = img.shape[3]
projector.ny = img.shape[2]
projector.nz = img.shape[1]
projector.dx = spacing[2]
projector.dy = spacing[1]
projector.dz = spacing[0]

for k in vars(projector):
    print (k, '=', getattr(projector, k))

In [ ]:
# setup the positions of projections, let's do 0, 45, 90, and 135
angles = np.array([0, 45, 90, 135]) * np.pi / 180

srcs = np.array([projector.dso * np.cos(angles), 
                 projector.dso * np.sin(angles), 
                 [0] * len(angles)]).T

det_centers = np.array([(projector.dso - projector.dsd) * np.cos(angles), 
                        (projector.dso - projector.dsd) * np.sin(angles), 
                        [0] * len(angles)]).T

det_us = np.array([-np.sin(angles), np.cos(angles), [0] * len(angles)]).T
det_vs = np.zeros_like(det_us)
det_vs[:, 2] = 1

In [ ]:
# very important: use np.copy('C') so that the arrays are saved in C order
cp.cuda.Device(0).use()
ct_projector.set_device(0)

cuimg = cp.array(img, cp.float32, order='C')
cusrcs = cp.array(srcs, cp.float32, order='C')
cudet_centers = cp.array(det_centers, cp.float32, order='C')
cudet_us = cp.array(det_us, cp.float32, order='C')
cudet_vs = cp.array(det_vs, cp.float32 ,order='C')

In [ ]:
# set which device to use
# projector.set_device(0)
# forward projection
cufp = ct_cone.siddon_fp_arbitrary(projector, cuimg, cudet_centers, cudet_us, cudet_vs, cusrcs)
fp = cufp.get()

In [ ]:
# show the projections
plt.figure(figsize = (16,4))
for i in range(4):
    plt.subplot(1,4,i+1)
    plt.imshow(fp[0, i, ...], 'gray')

In [ ]:
# backprojection: The siddon ray tracing has aliasing when backprojection. Need to be normalized
iangle = 2
cubp = ct_cone.siddon_bp_arbitrary(
    projector,
    cp.copy(cufp[:, [iangle], ...], 'C'), 
    cudet_centers[[iangle]], 
    cudet_us[[iangle]],
    cudet_vs[[iangle]],
    cusrcs[[iangle]]
)
cubp_norm = ct_cone.siddon_bp_arbitrary(
    projector,
    cp.ones(cufp[:, [iangle], ...].shape, cp.float32), 
    cudet_centers[[iangle]], 
    cudet_us[[iangle]], 
    cudet_vs[[iangle]], 
    cusrcs[[iangle]]
)
cubp = cubp / (cubp_norm + 1e-4)

In [ ]:
bp = cubp.get()

plt.figure(figsize = (12,4))
plt.subplot(131); plt.imshow(bp[0, bp.shape[1]//2, ...], 'gray', aspect=spacing[1] / spacing[2])
plt.subplot(132); plt.imshow(bp[0, :, bp.shape[2]//2, :], 'gray', aspect=spacing[0] / spacing[2])
plt.subplot(133); plt.imshow(bp[0, ..., bp.shape[3]//2], 'gray', aspect=spacing[0] / spacing[1])